In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19


2026-01-04 17:56:21.740706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767549381.925815     130 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767549381.985842     130 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
IMG_SIZE = 224

image_path = "/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity/images"
mask_path  = "/kaggle/input/covid19-radiography-database/COVID-19_Radiography_Dataset/Lung_Opacity/masks"

def load_data(img_dir, mask_dir):
    images, masks = [], []

    for file in sorted(os.listdir(img_dir)):
        img = cv2.imread(os.path.join(img_dir, file))
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        mask = cv2.imread(os.path.join(mask_dir, file), 0)
        mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
        mask = mask / 255.0
        mask = np.expand_dims(mask, axis=-1)

        images.append(img)
        masks.append(mask)

    return np.array(images), np.array(masks)

X, Y = load_data(image_path, mask_path)

# Train-validation split
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


In [4]:
def attention_gate(x, g, filters):
    """
    x: skip connection (higher resolution)
    g: gating signal (lower resolution)
    """

    # Upsample gating signal to match x
    g_up = UpSampling2D(
        size=(x.shape[1] // g.shape[1], x.shape[2] // g.shape[2]),
        interpolation="bilinear"
    )(g)

    theta_x = Conv2D(filters, 1, padding="same")(x)
    phi_g   = Conv2D(filters, 1, padding="same")(g_up)

    add = Add()([theta_x, phi_g])
    relu = Activation("relu")(add)

    psi = Conv2D(1, 1, padding="same")(relu)
    sigmoid = Activation("sigmoid")(psi)

    return Multiply()([x, sigmoid])


In [5]:
# def attention_gate(x, g, filters):
#     theta = Conv2D(filters, 1)(x)
#     phi   = Conv2D(filters, 1)(g)
#     add   = Add()([theta, phi])
#     relu  = Activation("relu")(add)
#     psi   = Conv2D(1, 1)(relu)
#     attn  = Activation("sigmoid")(psi)
#     return Multiply()([x, attn])


In [6]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, heads):
        super().__init__()
        self.att = MultiHeadAttention(num_heads=heads, key_dim=embed_dim)
        self.norm1 = LayerNormalization()
        self.norm2 = LayerNormalization()
        self.ffn = Sequential([
            Dense(embed_dim*4, activation="relu"),
            Dense(embed_dim)
        ])

    def call(self, x):
        attn = self.att(x, x)
        x = self.norm1(x + attn)
        ffn = self.ffn(x)
        return self.norm2(x + ffn)


In [7]:
def VGG19_Attn_TransUNet():

    inputs = Input((224,224,3))

    # ========== MODEL 1: VGG19 ==========
    vgg = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
    vgg.trainable = False

    s1 = vgg.get_layer("block1_conv2").output
    s2 = vgg.get_layer("block2_conv2").output
    s3 = vgg.get_layer("block3_conv4").output
    s4 = vgg.get_layer("block4_conv4").output
    b  = vgg.get_layer("block5_conv4").output

    # ========== MODEL 2: ATTENTION U-NET ==========
    att = attention_gate(s4, b, 512)
    up  = UpSampling2D()(b)
    d   = Concatenate()([up, att])
    d   = Conv2D(512, 3, padding="same", activation="relu")(d)

    # ========== MODEL 3: TRANSFORMER ==========
    shp = d.shape
    t = Reshape((-1, shp[-1]))(d)
    t = TransformerBlock(shp[-1], heads=8)(t)
    t = Reshape((shp[1], shp[2], shp[3]))(t)

    # Decoder
    up2 = UpSampling2D()(t)
    up2 = Concatenate()([up2, s3])
    up2 = Conv2D(256, 3, padding="same", activation="relu")(up2)

    up3 = UpSampling2D()(up2)
    up3 = Concatenate()([up3, s2])
    up3 = Conv2D(128, 3, padding="same", activation="relu")(up3)

    up4 = UpSampling2D()(up3)
    up4 = Concatenate()([up4, s1])
    up4 = Conv2D(64, 3, padding="same", activation="relu")(up4)

    outputs = Conv2D(1, 1, activation="sigmoid")(up4)

    return Model(inputs, outputs)


In [8]:
def dice_loss(y_true, y_pred):
    smooth = 1
    intersection = tf.reduce_sum(y_true * y_pred)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_coef(y_true, y_pred):
    smooth = 1
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)


In [9]:
model = VGG19_Attn_TransUNet()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=dice_loss,
    metrics=[dice_coef]
)

model.summary()

history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=40,
    batch_size=8
)


I0000 00:00:1767549553.590000     130 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


NameError: name 'Sequential' is not defined

In [ ]:
import matplotlib.pyplot as plt

pred = model.predict(X_val[:1])

plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
plt.title("Image")
plt.imshow(X_val[0])
plt.axis("off")

plt.subplot(1,3,2)
plt.title("Ground Truth")
plt.imshow(Y_val[0].squeeze(), cmap="gray")
plt.axis("off")

plt.subplot(1,3,3)
plt.title("Prediction")
plt.imshow(pred[0].squeeze(), cmap="gray")
plt.axis("off")
plt.show()
